In [1]:
import random

from distributed_processing.serializers import JsonSerializer
from distributed_processing.client import Client
from distributed_processing.redis_connector import RedisConnector


In [2]:
REDIS_HOST = "localhost"
REDIS_PORT = 6379
REDIS_DB = 0
NAMESPACE = "tasks"

In [3]:
redis_connector = RedisConnector(redis_host=REDIS_HOST, redis_port=REDIS_PORT, 
                                 redis_db=REDIS_DB, namespace=NAMESPACE)

client = Client(JsonSerializer(), redis_connector, check_registry="cache")

Client with id: tasks:redis_client:8
Results queue: tasks:redis_client:8:responses


In [4]:
y = client.rpc_async("add", [1, 0])

In [5]:
y.get()

1

In [6]:
client.check_registry ="allways"

try:
    y = client.rpc_async("fake", [1, 0])
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [7]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

try:
    y.get()
except Exception as e:
    print(e)


Error -32601 : The method does not exist/is not available.

 


In [8]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

y.safe_get(default="Esto es un error")

'Esto es un error'

In [9]:
client.check_registry ="cache"

In [10]:
x = client.rpc_async("div", [1, 0])

In [11]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32603 : Internal RPC error.

 Traceback (most recent call last):
  File "/home/augusto/tfm/distributed_processing/python/distributed_processing/worker.py", line 155, in _process_single_request
    result = func[request["method"]](*args, **kwargs)
  File "<ipython-input-4-119847af5757>", line 8, in div
    return x / y
ZeroDivisionError: division by zero



In [12]:
client.rpc_sync("add", [1, 1])

2

In [13]:
def f(x,y): return x + y

y = client.rpc_async_fn(f, [1, 2.0])

In [14]:
y.get()

3.0

In [15]:
client.rpc_sync_fn(f, [3.0, 2.0])

5.0

In [16]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)
    fs.append(client.rpc_async(t[0], t[1]))

('add', [0.9091075812107953, 0.04696808482934378], {})
('add', [0.23304927679876353, 0.378677128484019], {})
('add', [0.10228898210372384, 0.6891919091747304], {})
('div', [0.8318694376151906, 0.16326322168353813], {})
('mul', [0.7637391941538142, 0.7968327465404933], {})
('add', [0.7646976400709941, 0.6552168493793303], {})
('mul', [0.7985779086179677, 0.3470352997609364], {})
('div', [0.1732385545966293, 0.945310535090183], {})
('mul', [0.6147309712888489, 0.5416947499030671], {})
('add', [0.3576197313349665, 0.2028774897856157], {})


In [17]:
[f.get() for f in fs]

[0.956075666040139,
 0.6117264052827825,
 0.7914808912784542,
 5.095265357605449,
 0.6085723997182069,
 1.4199144894503244,
 0.2771347238996981,
 0.18326100066165268,
 0.3329965397499825,
 0.5604972211205822]

In [18]:
tp

[('add', [0.9091075812107953, 0.04696808482934378], {}),
 ('add', [0.23304927679876353, 0.378677128484019], {}),
 ('add', [0.10228898210372384, 0.6891919091747304], {}),
 ('div', [0.8318694376151906, 0.16326322168353813], {}),
 ('mul', [0.7637391941538142, 0.7968327465404933], {}),
 ('add', [0.7646976400709941, 0.6552168493793303], {}),
 ('mul', [0.7985779086179677, 0.3470352997609364], {}),
 ('div', [0.1732385545966293, 0.945310535090183], {}),
 ('mul', [0.6147309712888489, 0.5416947499030671], {}),
 ('add', [0.3576197313349665, 0.2028774897856157], {})]

In [19]:
fs = client.rpc_batch_async(tp)

In [20]:
[f.get() for f in fs]

[0.956075666040139,
 0.6117264052827825,
 0.7914808912784542,
 5.095265357605449,
 0.6085723997182069,
 1.4199144894503244,
 0.2771347238996981,
 0.18326100066165268,
 0.3329965397499825,
 0.5604972211205822]

In [21]:
client.rpc_batch_sync(tp)

[0.956075666040139,
 0.6117264052827825,
 0.7914808912784542,
 5.095265357605449,
 0.6085723997182069,
 1.4199144894503244,
 0.2771347238996981,
 0.18326100066165268,
 0.3329965397499825,
 0.5604972211205822]

In [22]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "fake"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)

('mul', [0.3946327028711428, 0.6245348725493295], {})
('add', [0.25322651648770567, 0.8720561101882707], {})
('fake', [0.9354624323824687, 0.5438217644348127], {})
('mul', [0.09614924643205969, 0.9031611633359359], {})
('div', [0.08219849565150694, 0.16681538306609023], {})
('div', [0.5550417849669401, 0.3606881947268552], {})
('add', [0.9594337489889184, 0.021774519440847295], {})
('mul', [0.5389812978767485, 0.09280279140271608], {})
('add', [0.9987943764613024, 0.24963096969691023], {})
('fake', [0.7157054340816611, 0.6751113108355713], {})


In [23]:
tp

[('mul', [0.3946327028711428, 0.6245348725493295], {}),
 ('add', [0.25322651648770567, 0.8720561101882707], {}),
 ('fake', [0.9354624323824687, 0.5438217644348127], {}),
 ('mul', [0.09614924643205969, 0.9031611633359359], {}),
 ('div', [0.08219849565150694, 0.16681538306609023], {}),
 ('div', [0.5550417849669401, 0.3606881947268552], {}),
 ('add', [0.9594337489889184, 0.021774519440847295], {}),
 ('mul', [0.5389812978767485, 0.09280279140271608], {}),
 ('add', [0.9987943764613024, 0.24963096969691023], {}),
 ('fake', [0.7157054340816611, 0.6751113108355713], {})]

In [24]:
client.check_registry ="never"
client.set_default_queue("cola_1")

fs = client.rpc_batch_async(tp)

In [25]:
[f.safe_get() for f in fs]

[0.2464618847914266,
 1.1252826266759763,
 None,
 0.08683826526145261,
 0.4927512927206533,
 1.5388410075003052,
 0.9812082684297657,
 0.05001896895682107,
 1.2484253461582127,
 None]

In [26]:
client.check_registry ="cache"

In [27]:
try:
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [28]:
try:    
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [29]:
client.check_registry="never"
client.set_default_queue("cola_1")

x = client.rpc_async("kk", [1, 0])

In [30]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 


In [31]:
y = client.rpc_async("add", [1, 0])

In [32]:
y.get(5)

1

In [33]:
def f(x,y): return x + y

client.check_registry="never"
y = client.rpc_async_fn(f, [1, 2.0])
try:
    y.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 
